##### 基礎演習 (30分)
* 畳み込み層 (L.Convolution2D)
* パディング・ストライドの指定 (pad, stride)
* プーリング層 (F.max\_pooling, F.avg\_pooling)
* 画像の入力
* 小課題：畳み込み・プーリングを使ってみる (使い方の学習)
* 小課題：パラメータ数の計算と比較
* Variableの形状変換 (F.flatten())

# Import

chainerを使ってコードを書くときによく使うものをまとめてimportしておきます

(Chainer Turorial推奨: https://docs.chainer.org/en/stable/tutorial/basic.html#core-concept 参照)

In [ ]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

# L.Convolution2D

2次元カーネルの畳み込み層は、chainerでは
```python
L.Convolution2D
```
を使います。

https://docs.chainer.org/en/stable/reference/generated/chainer.links.Convolution2D.html

（講義スライドでは3次元とか4次元とか書いてありますが、「フィルタ」は2次元なので、ここではConvolution2Dです）

`L.Convolution2D` の作用を確かめてみましょう。

(1, 1, 3, 3)次元の入力マップxに、(1, 1, 2, 2)次元のフィルタを畳み込んでみます。

講義スライドでは、「3次元入力に4次元フィルタを畳み込む」と書きましたが、
複数枚の画像をまとめて入力するため、Convolution2Dは4次元配列を入力に取るようになっています

In [ ]:
x_data = np.array([[[
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
]]], dtype=np.float32)
x = Variable(x_data)

In [ ]:
conv = L.Convolution2D(
    in_channels=1, out_channels=1, ksize=2, stride=1, pad=0,
    initialW=np.array([[[
        [1, 2],
        [3, 4],
    ]]]),
    initial_bias=np.array([0])
)

In [ ]:
conv(x)

手で計算してみて、結果を見比べてみると分かりやすいかもしれません

例えば、[0,0]は `1*1 + 2*2 + 3*4 + 4*5 = 37` になっています

# 画像入力

In [ ]:
from skimage.io import imread
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
lena = imread('./lena_gray.gif')

In [ ]:
plt.imshow(lena, cmap='gray')

画像はピクセルの配列です。

In [ ]:
lena

In [ ]:
lena.shape

エッジ検出フィルタを畳み込んでみます

In [ ]:
vertical_edge = L.Convolution2D(
    in_channels=1, out_channels=1, ksize=3, stride=1, pad=0,
    initialW=np.array([[[
        [0, 0, 0],
        [-0.5, 0, 0.5],
        [0, 0, 0]
    ]]]),
    initial_bias=np.array([0])
)

In [ ]:
lena_var = Variable(np.array([[lena]], dtype=np.float32))

In [ ]:
result = vertical_edge(lena_var)

In [ ]:
plt.imshow(result.data[0][0], cmap='gray')